# **온라인 상점 구축 프로젝트**

## **1. 환경 구축하기**

- 가상환경 설정

In [ ]:
python -m venv myshop
cd pystagram
source ./bin/activate
# Windows의 경우 .\Scripts\activate

- 필수 라이브러리 설치

In [ ]:
pip install -r requirements.txt

## **2. 온라인 상점 기반 구축**

- 주요 기능
    - 제품 카탈로그 만들기 → shop
    - 광고 세션을 사용한 쇼핑 카트 구축 → cart
    - 고객 주문관리 → order

### 2.1 제품 카탈로그 만들기

- 온라인 상점(쇼핑몰)은 실제 상품이 진열되는 것이 아니라 제품 카탈로그를 기반으로 구성됨
- 즉 제품 카탈로그를 만드는 작업이 온라인 상점의 기본 틀을 만드는 작업과 동일하다고 할 수 있음

#### 2.1.1 기초 작업

- 프로젝트 생성

In [ ]:
django-admin startproject myshop .

- 앱 생성

In [ ]:
django-admin startapp shop

- 앱 등록(config/settings.py)

In [ ]:
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'shop.apps.ShopConfig',
]

#### 2.1.2 제품 카탈로그 모델 생성

- 다양한 카테고리의 제품으로 구성됨
- 각 제품에는 이름, 설명(선택사항), 이미지(선택사항), 가격, 사용 가능성 등이 포함됨


- shop/models.py

In [ ]:
from django.db import models
from django.urls import reverse

class Category(models.Model):
    name = models.CharField(max_length=200)
    slug = models.SlugField(max_length=200, unique=True)

    class Meta:
        ordering = ['name']
        indexes = [
            models.Index(fields=['name']),
        ]
        verbose_name = 'category'
        verbose_name_plural = 'categories'

    def __str__(self):
        return self.name

    def get_absolute_url(self):
        return reverse('shop:product_list_by_category', args=[self.slug])


class Product(models.Model):
    category = models.ForeignKey(Category, related_name='products', on_delete=models.CASCADE)
    name = models.CharField(max_length=200)
    slug = models.SlugField(max_length=200)
    image = models.ImageField(upload_to='products/%Y/%m/%d', blank=True)
    description = models.TextField(blank=True)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    available = models.BooleanField(default=True)
    created = models.DateTimeField(auto_now_add=True)
    updated = models.DateTimeField(auto_now=True)

    class Meta:
        ordering = ['name']
        indexes = [
            models.Index(fields=['id', 'slug']),
            models.Index(fields=['name']),
            models.Index(fields=['-created']),
        ]

    def __str__(self):
        return self.name

    def get_absolute_url(self):
        return reverse('shop:product_detail', args=[self.id, self.slug])

- Terminal

In [ ]:
python manage.py makemigrations
python manage.py migrate

#### 2.1.3 관리사이트(Admin)에 카탈로그 모델 등록하기

In [ ]:
from django.contrib import admin
from .models import Category, Product


@admin.register(Category)
class CategoryAdmin(admin.ModelAdmin):
    list_display = ['name', 'slug']
    prepopulated_fields = {'slug': ('name',)}


@admin.register(Product)
class ProductAdmin(admin.ModelAdmin):
    list_display = ['name', 'slug', 'price', 'available', 'created', 'updated']
    list_filter = ['available', 'created', 'updated']
    list_editable = ['price', 'available']
    prepopulated_fields = {'slug': ('name',)}

- Terminal

In [ ]:
python manage.py createsuperuser
python manage.py runserver

#### 2.1.4 카탈로그 뷰 만들기

- shop/views.py

In [ ]:
from django.shortcuts import render, get_object_or_404
from .models import Category, Product


def product_list(request, category_slug=None):
    category = None
    categories = Category.objects.all()
    products = Product.objects.filter(available=True)
    if category_slug:
        category = get_object_or_404(Category, slug=category_slug)
        products = products.filter(category=category)

    return render(request, 'shop/product/list.html',
                  {'category': category, 'categories': categories, 'products': products})


def product_detail(request, id, slug):
    product = get_object_or_404(Product, id=id, slug=slug, available=True)

    return render(request, 'shop/product/detail.html',
                  {'product': product})

- shop/urls.py

In [ ]:
from django.urls import path
from . import views

app_name = 'shop'

urlpatterns = [
    path('', views.product_list, name='product_list'),
    path('<slug:category_slug>/', views.product_list, name='product_list_by_category'),
    path('<int:id>/<slug:slug>/', views.product_detail, name='product_detail'),
]

- myshop/urls.py

In [ ]:
from django.contrib import admin
from django.urls import path, include


urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('shop.urls', namespace='shop')),
]

#### 2.1.5 카탈로그 템플릿 생성하기

- shop/templates/shop/base.html
    - 제품 목록 및 상세 페이지용 베이스 템플릿

In [ ]:
{% load static %}
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8" />
    <title>{% block title %}My shop{% endblock %}</title>
    <link href="{% static "css/base.css" %}" rel="stylesheet">
</head>
<body>
    <div id="header">
        <a href="/" class="logo">My shop</a>
    </div>
    <div id="subheader">
        <div class="cart">
            {% with total_items=cart|length %}
                {% if total_items > 0 %}
                    Your cart:
                    <a href="{% url "cart:cart_detail" %}">
                        {{ total_items }} item{{ total_items|pluralize }},
                        ${{ cart.get_total_price }}
                    </a>
                {% elif not order %}
                    Your cart is empty.
                {% endif %}
            {% endwith %}
        </div>
    </div>
    <div id="content">
        {% block content %}
        {% endblock %}
    </div>
</body>
</html>

- shop/product/list.html
    - 제품 목록 템플릿

In [ ]:
{% extends "shop/base.html" %}
{% load static %}

{% block title %}
  {% if category %}{{ category.name }}{% else %}Products{% endif %}
{% endblock %}

{% block content %}
  <div id="sidebar">
    <h3>Categories</h3>
    <ul>
      <li {% if not category %}class="selected"{% endif %}>
        <a href="{% url "shop:product_list" %}">All</a>
      </li>
      {% for c in categories %}
        <li {% if category.slug == c.slug %}class="selected"{% endif %}>
          <a href="{{ c.get_absolute_url }}">{{ c.name }}</a>
        </li>
      {% endfor %}
    </ul>
  </div>
  <div id="main" class="product-list">
    <h1>{% if category %}{{ category.name }}{% else %}Products{% endif %}</h1>
    {% for product in products %}
      <div class="item">
        <a href="{{ product.get_absolute_url }}">
          <img src="{% if product.image %}{{ product.image.url }}{% else %}{% static "img/no_image.png" %}{% endif %}">
        </a>
        <a href="{{ product.get_absolute_url }}">{{ product.name }}</a>
        <br>
        ${{ product.price }}
      </div>
    {% endfor %}
  </div>
{% endblock %}

- myshop/settings.py

In [ ]:
MEDIA_URL = 'media/'
MEDIA_ROOT = BASE_DIR / 'media'

- myshop/urls.py

In [ ]:
from django.contrib import admin
from django.urls import path, include
from django.conf import settings
from django.conf.urls.static import static


urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('shop.urls', namespace='shop')),
]

if settings.DEBUG:
    urlpatterns += static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)

- shop/product/detail.html

In [ ]:
{% extends "shop/base.html" %}
{% load static %}

{% block title %}
  {{ product.name }}
{% endblock %}
{% block content %}
  <div class="product-detail">
    <img src="{% if product.image %}{{ product.image.url }}{% else %}
    {% static "img/no_image.png" %}{% endif %}">
    <h1>{{ product.name }}</h1>
    <h2>
      <a href="{{ product.category.get_absolute_url }}">
        {{ product.category }}
      </a>
    </h2>
    <p class="price">${{ product.price }}</p>
    {{ product.description|linebreaks }}
  </div>
{% endblock %}

### 2.2. 쇼핑 카트 만들기

- 제품 카탈로그를 기반으로 사용자가 구매하려는 제품을 선택할 수 있도록 쇼핑 카트(장바구니) 만들기
- 장바구니를 사용하면 사용자가 사이트를 탐색하는 동안 최종 주문이 이루어질 때까지 제품을 선택하고 주문한 금액을 설정한 정보를 임시로 저장할 수 있음
- 사용자가 방문하는 동안, 카트 항목이 유지되도록 카트는 세션에서 유지되어야 함

#### 2.2.1 세션

- 세션(Session)
    - 일정 시간 동안 같은 사용자(브라우저)로부터 들어오는 일련의 요구를 하나의 상태로 보고, 그 상태를 유지시키는 기술
    - 일정 시간: 방문자가 웹 브라우저를 통해 웹 서버에 접속한 시점부터 웹 브라우저를 종료하여 연결을 끝내는 시점까지를 가리킴
    - 즉, 방문자가 웹 서버에 접속해 있는 상태를 하나의 단위로 보고 그것을 세션이라고 함
    <br><br>
- 세션의 특징
    - 웹 서버에 웹 컨테이너의 상태를 유지하기 위한 정보를 저장
    - 웹 서버에 저장되는 쿠키(=세션 쿠키)
    - 브라우저를 닫거나, 서버에서 세션을 삭제했을 때만 삭제가 되므로, 쿠키보다 비교적 보안성이 좋음
    - 저장 데이터에 제한이 없음(서버 용량이 허용하는 한에서)
    - 각 클라이언트에 고유 Session ID를 부여하고 Session ID로 클라이언트를 구분해 각 요구에 맞는 서비스를 제공
    <br><br>
    - 세션과 쿠키
        - 서로 유사한 역할은 수행하는 기술
        - 세션은 서버의 자원을 사용하며 쿠키는 클라이언트의 자원을 사용함
    <br><br>
- 세션의 동작 순서
    - 클라이언트가 페이지에 요청
    - 서버는 접근한 클라이언트의 Request-Header 필드인 Cookie를 확인하여, 클라이언트가 해당 session-id를 보냈는지 확인
    - session-id가 존재하지 않는다면 서버는 session-id를 생성해 클라이언트에게 전달
    - 클라이언트는 서버로부터 받은 session-id를 쿠키에 저장
    - 클라이언트는 서버에 요청시 이 쿠키의 session-id 값을 같이 서버에 전달
    - 서버는 전달받은 session-id로 session에 있는 클라이언트 정보를 가지고 요청을 처리 후 응답
    <br><br>
- 사용 예시
    - 화면을 이동해도 로그인이 풀리지 않고 로그아웃하기 전까지 유지

#### 2.2.2 장고 세션 프레임워크

- 장고 세션 프레임워크
    - 익명 세션과 사용자 세션 지원
    - 세션 프레임워크를 통해 각 방문자의 데이터를 저장할 수 있음

- 장고 세션 미들웨어
    - 쿠키의 송수신을 관리함

- 기본 세션엔진
    - 세션 데이터를 데이터베이스에 저장
    - 요구에 따라 다른 세션 엔진을 선택할 수 있음
<br><br>
- 장고 세션의 사용
    - 프로젝트의 settings.py에서 MIDDLEWARE 설정에 'django.contrib.sessions.middleware.SessionMiddleware' 포함여부 확인
    - request.session을 사용하여 현재 세션에 접근 가능
    - 세션 데이터를 저장하고 검색할 때에는 파이썬의 딕셔너리처럼 취급함
    - 세션 딕셔너리는 기본적으로 JSON으로 직렬화할 수 있는 모든 파이썬 객체를 허용함
    ```
    # 세션 변수 설정
    request.session['foo'] = 'bar'

    # 세션 키 조회
    request.session.get['foo']

    # 세션에 저장한 키 삭제
    del request.session['foo']
    ```

#### 2.2.3 세션 설정


- 세션 설정
    - 세션 구성에 사용되는 설정
    - 가장 중요한 것은 SESSION_ENGINE: 세션이 저장되는 위치를 설정할 수 있음
    - 장고의 기본 세션 모델: django.contrib.sessions
    - 장고 세션의 옵션
        - 데이터베이스 세션: 기본 세션 엔진. 세션 데이터가 데이터베이스에 저장됨
        - 파일 기반 세션
        - 캐시기반 세션
        - 캐시 데이터베이스 세션
        - 쿠키 기반 세션

- 세션 커스터마이징을 위한 설정들
    - SESSION_COOKIE_AGE
        - 세션 쿠키의 유효기간(초). 기본 값은 1209600(2주일)
    - SESSION_COOKIE_DOMAIN
        - 세션 쿠키에 사용되는 도메인
            - 크로스 도메인 쿠키 사용: mydomain.com으로 설정
            - 표준 도메인 쿠키 사용: None으로 설정
    - SESSION_COOKIE_HTTPONLY
        - 세션 쿠키에 HttpOnly 플래그 사용 여부 지정
        - True로 설정하면 클라이언트의 자바스크립트에서 세션 쿠키에 접근할 수 없음
        - 기본 값은 True: 사용자 세션 탈취(Hijacking)에 대한 보안 강화 목적
    - SESSION_COOKIE_SECURE
        - HTTPS 연결인 경우에만 쿠키를 전송할 수 있음
        - 기본 값은 False
    - SESSION_EXPIRE_AT_BROWSER_CLOSE
        - 브라우저를 닫을 때 세션을 만료해야 함
        - 기본 값은 False
    - SESSION_SAVE_EVERY_REQUEST
        - True인 경우, 요청할 때마다 세션을 데이터베이스에 저장
        - 세션이 저장될 때마다 세션 만료 시점도 갱신됨
        - 기본 값은 False


#### 2.2.4 세션 만료

- SESSION_EXPIRE_AT_BROWSER_CLOSE
    - 설정의 사용으로 브라우저 기간 세션 또는 영구 세션의 사용을 설정할 수 있음
    - SESSION_COOKIE_AGE
    - 기본 값은 False
        - False: 세션 지속 시간을 SESSION_COOKIE_AGE에 설정된 값으로 강제 설정
        - True: 사용자가 브라우저를 닫을 때 세션이 만료되며 SESSION_COOKIE_AGE 설정은 무시됨
- request.session.set_expiry() 메서드를 사용하여 현재 세션의 기간을 덮어쓸 수 있음


#### 2.2.5 세션에 쇼핑 카트 저장하기

- 세션에 카트 아이템들을 저장하려면 JSON으로 직렬화할 수 있는 간단한 구조를 만들어야 함
    1. 카트에는 카트에 포함된 각 품목에 대한 다음의 데이터가 포함되어야 함
        - Product 인스턴스의 ID
        - 선택한 제품의 수량
        - 제품의 단가
    2. 쇼핑 카트 생성
    3. 쇼핑 카트와 세션의 연결 기능 구축
        - 카트가 필요한 경우 사용자의 세션키가 설정되어 있는지 체크
            - 설정된 키가 없을 경우 새로운 카트를 생성하고 카트 세션 키에 저장
        - 이후 요청 시 동일한 확인을 수행해서 카트 세션 키에서 카트 아이템들을 가져옴
        - 세션에서 카트 아이템들을 조회
        - 관련 Product 객체들을 데이터베이스에서 조회

- myshop/settings.py

In [ ]:
CART_SESSION_ID = 'cart'

- Terminal
    - cart 앱 생성

In [ ]:
python manage.py startapp cart

- myshop/settings.py

In [ ]:
INSTALLED_APPS = [
    ...
    'shop.apps.ShopConfig',
    'cart.apps.CartConfig',
]

- cart/cart.py
    - 카트 초기화를 위한 __ init__() 정의
    - 카트에 아이템을 추가하기 위한 add(), save() 정의
        - add()의 매개변수
            - product: 카트에 추가하거나 업데이트할 제품의 인스턴스
            - quantity: 제품의 수량. 기본 값은 1
            - override_quantity: 지정된 수량으로 수량을 재정의해야 하는지(True) 또는 기존 수량에 새로운 수량을 추가해야 하는지(False)를 나타내는 Boolean 값
    - 카트의 아이템을 제거하기 위한 remove() 정의

In [ ]:
from decimal import Decimal
from django.conf import settings
from shop.models import Product

class Cart:
    def __init__(self, request):
        self.session = request.session
        cart = self.session.get(settings.CART_SESSION_ID)
        if not cart:
            cart = self.session[settings.CART_SESSION_ID] = {}
        self.cart = cart

    def add(self, product, quantity=1, override_quantity=False):
        product_id = str(product.id)
        if product_id not in self.cart:
            self.cart[product_id] = {'quantity': 0, 'price': str(product.price)}
        if override_quantity:
            self.cart[product_id]['quantity'] = quantity
        else:
            self.cart[product_id]['quantity'] += quantity
        self.save()

    def save(self):
        self.session.modified = True

    def remove(self, product):
        product_id = str(product.id)
        if product_id in self.cart:
            del self.cart[product_id]
            self.save()

- cart/cart.py
    - 카트에 포함된 아이템들을 반복해서 관련 Product 인스턴스에 접근해야 함. 이를 위해서 __ iter__()정의
    - 카트에 포함된 아이템들의 총 수를 반환하기 위하여 __ len__() 정의

In [ ]:
    def __iter__(self):
        product_ids = self.cart.keys()
        # get the product objects and add them to the cart
        products = Product.objects.filter(id__in=product_ids)
        cart = self.cart.copy()
        for product in products:
            cart[str(product.id)]['product'] = product
        for item in cart.values():
            item['price'] = Decimal(item['price'])
            item['total_price'] = item['price'] * item['quantity']
            yield item

    def __len__(self):
        return sum(item['quantity'] for item in self.cart.values())

- cart/cart.py
    - 모든 카트 아이템들의 수량의 합을 반환하기 위하여 get_total_price() 정의
    - 카트 세션을 지우기 위한 clear() 정의

In [ ]:
    def get_total_price(self):
        return sum(Decimal(item['price']) * item['quantity'] for item in self.cart.values())

    def clear(self):
        # remove cart from session
        del self.session[settings.CART_SESSION_ID]
        self.save()

#### 2.2.6 카트 관련 뷰 만들기

- cart/forms.py
    - 카트에 아이템 추가하기

In [ ]:
from django import forms


PRODUCT_QUANTITY_CHOICES = [(i, str(i)) for i in range(1, 21)]


class CartAddProductForm(forms.Form):
    quantity = forms.TypedChoiceField(
                                choices=PRODUCT_QUANTITY_CHOICES,
                                coerce=int)
    override = forms.BooleanField(required=False,
                                  initial=False,
                                  widget=forms.HiddenInput)

- cart/views.py

In [ ]:
from django.shortcuts import render, redirect, get_object_or_404
from django.views.decorators.http import require_POST
from shop.models import Product
from .cart import Cart
from .forms import CartAddProductForm


@require_POST
def cart_add(request, product_id):
    cart = Cart(request)
    product = get_object_or_404(Product, id=product_id)
    form = CartAddProductForm(request.POST)
    if form.is_valid():
        cd = form.cleaned_data
        cart.add(product=product,
                 quantity=cd['quantity'],
                 override_quantity=cd['override'])
    return redirect('cart:cart_detail')


@require_POST
def cart_remove(request, product_id):
    cart = Cart(request)
    product = get_object_or_404(Product, id=product_id)
    cart.remove(product)
    return redirect('cart:cart_detail')


def cart_detail(request):
    cart = Cart(request)
    for item in cart:
        item['update_quantity_form'] = CartAddProductForm(initial={
                            'quantity': item['quantity'],
                            'override': True})
    return render(request, 'cart/detail.html', {'cart': cart})

- cart/urls.py

In [ ]:
from django.urls import path
from . import views

app_name = 'cart'

urlpatterns = [
    path('', views.cart_detail, name='cart_detail'),
    path('add/<int:product_id>/', views.cart_add, name='cart_add'),
    path('remove/<int:product_id>/', views.cart_remove, name='cart_remove'),
]

- cart/templates/cart/detail.html
    - 카트를 표시하기 위한 템플릿 작성

In [ ]:
{% extends "shop/base.html" %}
{% load static %}

{% block title %}
  Your shopping cart
{% endblock %}

{% block content %}
  <h1>Your shopping cart</h1>
  <table class="cart">
    <thead>
      <tr>
        <th>Image</th>
        <th>Product</th>
        <th>Quantity</th>
        <th>Remove</th>
        <th>Unit price</th>
        <th>Price</th>
      </tr>
    </thead>
    <tbody>
      {% for item in cart %}
        {% with product=item.product %}
          <tr>
            <td>
              <a href="{{ product.get_absolute_url }}">
                <img src="{% if product.image %}{{ product.image.url }}
                {% else %}{% static "img/no_image.png" %}{% endif %}">
              </a>
            </td>
            <td>{{ product.name }}</td>
            <td>{{ item.quantity}}</td>
            <td>
              <form action="{% url "cart:cart_remove" product.id %}" method="post">
                <input type="submit" value="Remove">
                {% csrf_token %}
              </form>
            </td>
            <td class="num">${{ item.price }}</td>
            <td class="num">${{ item.total_price }}</td>
          </tr>
        {% endwith %}
      {% endfor %}
      <tr class="total">
        <td>Total</td>
        <td colspan="4"></td>
        <td class="num">${{ cart.get_total_price }}</td>
      </tr>
    </tbody>
  </table>
  <p class="text-right">
    <a href="{% url "shop:product_list" %}" class="button
    light">Continue shopping</a>
    <a href="{% url "orders:order_create" %}" class="button">Checkout</a>
  </p>
{% endblock %}

- shop/views.py
    - 카트에 제품 추가하기

In [ ]:
from cart.forms import CartAddProductForm

...
def product_detail(request, id, slug):
    product = get_object_or_404(Product, id=id, slug=slug, available=True)
    cart_product_form = CartAddProductForm()
    return render(request,
                  'shop/product/detail.html',
                  {'product': product,
                   'cart_product_form': cart_product_form})

- shop/templates/shop/product/detail.html

In [ ]:
...
    <p class="price">${{ product.price }}</p>
    <form action="{% url "cart:cart_add" product.id %}" method="post">
      {{ cart_product_form }}
      {% csrf_token %}
      <input type="submit" value="Add to cart">
    </form>
    {{ product.description|linebreaks }}
...

- cart/views.py
    - 카트에 제품 수량 업데이트하기

In [ ]:
...
def cart_detail(request):
    cart = Cart(request)
    for item in cart:
        item['update_quantity_form'] = CartAddProductForm(initial={
                            'quantity': item['quantity'],
                            'override': True})
    return render(request, 'cart/detail.html', {'cart': cart})

- cart/templates/cart/detail.html

In [ ]:
# <td>{{ item.quantity}}</td>를 찾아서 수정
...
<td>
    <form action="{% url "cart:cart_add" product.id %}" method="post">
    {{ item.update_quantity_form.quantity }}
    {{ item.update_quantity_form.override }}
    <input type="submit" value="Update">
    {% csrf_token %}
    </form>
</td>
...

In [ ]:
from django.shortcuts import render, redirect, get_object_or_404
from django.views.decorators.http import require_POST
from shop.models import Product
from .cart import Cart
from .forms import CartAddProductForm


@require_POST
def cart_add(request, product_id):
    cart = Cart(request)
    product = get_object_or_404(Product, id=product_id)
    form = CartAddProductForm(request.POST)
    if form.is_valid():
        cd = form.cleaned_data
        cart.add(product=product,
                 quantity=cd['quantity'],
                 override_quantity=cd['override'])
    return redirect('cart:cart_detail')


@require_POST
def cart_remove(request, product_id):
    cart = Cart(request)
    product = get_object_or_404(Product, id=product_id)
    cart.remove(product)
    return redirect('cart:cart_detail')


def cart_detail(request):
    cart = Cart(request)
    for item in cart:
        item['update_quantity_form'] = CartAddProductForm(initial={
                            'quantity': item['quantity'],
                            'override': True})
    return render(request, 'cart/detail.html', {'cart': cart})

#### 2.2.7 현재 카트에 대한 콘텍스트 프로세스 생성하기

### 2.3 고객 주문 등록하기

## **3. 비동기 작업**